In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 🧮 Função para calcular os parâmetros logísticos
def calcular_parametros(demanda_diaria, desvio_padrao_diario, lead_time_dias, custo_pedido, custo_armazenagem, preco_unitario):
    z = 1.65  # 95% de nível de serviço
    es = z * desvio_padrao_diario * np.sqrt(lead_time_dias)
    demanda_anual = demanda_diaria * 250
    lec = np.sqrt((2 * demanda_anual * custo_pedido) / (custo_armazenagem * preco_unitario))
    intervalo = lec / demanda_diaria
    estoque_max = es + (demanda_diaria * lead_time_dias)

    if lec >= estoque_max:
        estoque_medio = es + ((demanda_diaria * lead_time_dias) / 2)
    else:
        estoque_medio = es + (lec / 2)

    return {
        "Estoque de Segurança": round(es, 2),
        "LEC (Lote Econômico)": round(lec, 2),
        "Intervalo entre Pedidos (dias)": round(intervalo, 2),
        "Estoque Máximo": round(estoque_max, 2),
        "Estoque Médio": round(estoque_medio, 2)
    }

# 📊 Função para simular a curva de estoque com linhas de referência
def simular_curva_estoque(lec, es, demanda_diaria, estoque_medio, estoque_max):
    dias = list(range(1, int(2 * lec / demanda_diaria) + 1))
    estoque = []
    nivel = lec + es  # estoque inicial

    for dia in dias:
        if dia % round(lec / demanda_diaria) == 0:
            nivel += lec
        nivel -= demanda_diaria
        estoque.append(nivel)

    # Define o maior valor do eixo Y com base no maior parâmetro (LEC, EMAX ou ponto máximo da curva)
    margem = 0.15
    topo_y = max(nivel + lec, estoque_max, max(estoque)) * (1 + margem)

    # Geração do gráfico
    plt.figure(figsize=(10, 5))
    plt.plot(dias, estoque, marker='o', label='Estoque Diário')
    plt.axhline(y=es, color='red', linestyle='--', label='Estoque de Segurança')
    plt.axhline(y=estoque_medio, color='orange', linestyle='--', label='Estoque Médio')
    plt.axhline(y=estoque_max, color='green', linestyle='--', label='Estoque Máximo')

    plt.title('Simulação da Curva de Estoque com Faixas de Referência')
    plt.xlabel('Dias')
    plt.ylabel('Nível de Estoque')
    plt.ylim(0, topo_y)
    plt.legend()
    plt.grid(True)
    plt.savefig("grafico_estoque.png")
    plt.show()

# 🏁 Execução do exemplo prático
if __name__ == "__main__":
    resultado = calcular_parametros(
        demanda_diaria=40,
        desvio_padrao_diario=8,
        lead_time_dias=5,
        custo_pedido=120,
        custo_armazenagem=0.20,
        preco_unitario=30
    )

    print("📋 Parâmetros Calculados:")
    for chave, valor in resultado.items():
        print(f"- {chave}: {valor}")

    simular_curva_estoque(
        lec=resultado["LEC (Lote Econômico)"],
        es=resultado["Estoque de Segurança"],
        demanda_diaria=40,
        estoque_medio=resultado["Estoque Médio"],
        estoque_max=resultado["Estoque Máximo"]
    )
